### Imports

In [72]:
import requests
import json
import ee
import geemap
import folium
import os
import pandas as pd
import glob
import time
import math
import numpy as np

from datetime import datetime
#to change to be explicit
from modules.area_stats import *
from modules.agstack_to_gee import * 
from modules.tidy_tables import * 
from modules.WDPA_prep import *

import modules.area_stats as area_stats
import modules.WDPA_prep as WDPA_prep
from modules.agstack_to_gee import geo_id_list_to_feature_collection 
from modules.tidy_tables import tidy_dataframe_after_pivot 


# from modules.random_polys import *
# import importlib
# importlib.reload(area_stats)
print("imports complete")

imports complete


### Web app deployed URLs

In [73]:
asset_registry_base = "https://api-ar.agstack.org"
user_registry_base = "https://user-registry.agstack.org"

### Add your email & password to register with Agstack and test out the APIs

In [74]:
email = "my_name@domain.org"
phone_num = "0123456789"
password = "this_is_not_a_password"
discoverable = True
##appears to not need proper credentials for running this...

#### Start Session

NB this is timing out so skipping section - seems to work without somehow. Maybe an open connection already...

In [75]:
# ##using session to store cookies that are persistent
# session = requests.session()
# session.headers = headers = {
#     'Accept': 'application/json',
#     'Content-Type': 'application/json'
# }
# req_body = {'email': email, 'password': password}
# res = session.post(user_registry_base, json=req_body)
# print(session.cookies)
# print(res.status_code)

In [76]:
ee.Initialize()

#### Parameters

In [77]:
out_path = os.path.join('/home/sepal-user/fdap/')

#long csv
out_long_csv_name = 'stats_long_format_continuous_data.csv' 

dataset_column_name = "dataset_name"

#wide csv
out_file_long=out_path+out_long_csv_name

out_wide_csv_name = 'stats_wide_format_continuous_data.csv' #set output name

out_file_wide = out_path+out_wide_csv_name #set full path for output csv


In [137]:
#output column names
geometry_area_column = "Geometry_area_km2"

geo_id_column = "Geo_id"

#debug printing
verbose = True

## stats by pixel_area
to_pixel_area = True

## reducer choice
reducer_choice = ee.Reducer.sum().combine(
  reducer2=ee.Reducer.count(), sharedInputs=True)

#ee.Reducer.sum()

Local deforestation alert parameters

In [138]:
# Define how many days back
how_many_days_back = -(365*2)  # must be negative

#define buffer distance (m)
local_alerts_buffer_radius = 2000 

For defining random polygons (TESTING ONLY - will tidy if needed)

In [139]:
#if true will make number_of_points per admin boundary (i.e. no of points x no of boundaries), as opposed tob spread across them
points_per_admin_boundary = False 

#how many random admin boundaries do we select
number_of_boundaries = 20

#how many points  
number_of_points = 30

#buffer
buffer_distance_meters = 500

#max error in m (coarse is quicker to run) 
max_error_alert_buff = 100 

seed = 20 # so it can be reproducable - each number is a new random combination                                                                                                
   
max_error = 1000 #in meters for admin boundaries (can be coarse ot speed up computation, as just example)


### 1. Grab land cover datasets

#### Fetch: GLAD: Global 2000-2020 Land Cover and Land Use Change

For full legend see: https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/legend.xlsx

NB all tree cover have values related to height: from 3m to >25m. 
stable tree height values: 25-48 (terra firma); 125-148 (wetland);
height after disturbance values: 49-72 (terra firma); 149-172 (wetland) 
tree gain height values: 73-96 (terra firma); 173-196 (wetland); 

for now including all heights until agreed forest definition 

- value 25 tree stable
- value 49 tree after disturbance 
- value 73 tree gain
- value 248 tree from crop (NB need to investigate: likely plantation but could be secondary)
- value 245 crop
- value 24 vegetation 
- value 200 water 
- value 250 built up


In [140]:
glad_landcover_2020 = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020')
landmask = ee.Image("projects/glad/OceanMask").lte(1)
glad_landcover_2020 = glad_landcover_2020.updateMask(landmask);

#visualisation parameters (for original land cover)
visParamMap = {"min":0,"max":255,"palette":["FEFECC","FAFAC3","F7F7BB","F4F4B3","F1F1AB","EDEDA2","EAEA9A","E7E792","E4E48A",
"E0E081","DDDD79","DADA71","D7D769","D3D360","D0D058","CDCD50","CACA48","C6C63F","C3C337","C0C02F","BDBD27","B9B91E","B6B616",
"B3B30E","B0B006","609C60","5C985C","589558","549254","508E50","4C8B4C","488848","448544","408140","3C7E3C","387B38","347834",
"317431","2D712D","296E29","256B25","216721","1D641D","196119","155E15","115A11","0D570D","095409","065106","643700","643a00",
"643d00","644000","644300","644600","644900","654c00","654f00","655200","655500","655800","655a00","655d00","656000","656300",
"666600","666900","666c00","666f00","667200","667500","667800","667b00","ff99ff","FC92FC","F98BF9","F685F6","F37EF3","F077F0",
"ED71ED","EA6AEA","E763E7","E45DE4","E156E1","DE4FDE","DB49DB","D842D8","D53BD5","D235D2","CF2ECF","CC27CC","C921C9","C61AC6",
"C313C3","C00DC0","BD06BD","bb00bb","000003","000004","000005","BFC0C0","B7BDC2","AFBBC4","A8B8C6","A0B6C9","99B3CB","91B1CD",
"89AFD0","82ACD2","7AAAD4","73A7D6","6BA5D9","64A3DB","5CA0DD","549EE0","4D9BE2","4599E4","3E96E6","3694E9","2E92EB","278FED",
"1F8DF0","188AF2","1088F4","0986F7","55A5A5","53A1A2","519E9F","4F9B9C","4D989A","4B9597","499294","478F91","458B8F","43888C",
"418589","3F8286","3D7F84","3B7C81","39797E","37767B","357279","336F76","316C73","2F6970","2D666E","2B636B","296068","285D66",
"bb93b0","B78FAC","B48CA9","B189A6","AE85A2","AA829F","A77F9C","A47B99","A17895","9E7592","9A718F","976E8C","946B88","916885",
"8D6482","8A617F","875E7B","845A78","815775","7D5472","7A506E","774D6B","744A68","714765","de7cbb","DA77B7","D772B3","D46EAF",
"D169AB","CE64A8","CB60A4","C85BA0","C4579C","C15298","BE4D95","BB4991","B8448D","B54089","B23B86","AF3682","AB327E","A82D7A",
"A52976","A22473","9F1F6F","9C1B6B","991667","961264","000000","000000","000000",
"1964EB","1555E4","1147DD","0E39D6","0A2ACF","071CC8","030EC1","0000BA",
"0000BA","040464","0000FF","3051cf","000000","000000","000000","000000",
"000000","000000","000000","000000","000000","000000","000000","000000",
"000000","000000","000000","000000","000000","000000","000000","000000",
"547FC4","4D77BA","466FB1","4067A7","395F9E","335895","335896","335897","ff2828","ffffff","d0ffff","ffe0d0","ff7d00","fac800","c86400",
"fff000","afcd96","afcd96","64dcdc","00ffff","00ffff","00ffff","111133","000000"]};

#trees
glad_landcover_2020_main = glad_landcover_2020.where((glad_landcover_2020.gte(25)).And(glad_landcover_2020.lte(48)), 25)
glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(125)).And(glad_landcover_2020.lte(148)), 25)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(49)).And(glad_landcover_2020.lte(72)), 49)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(149)).And(glad_landcover_2020.lte(172)), 49)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(73)).And(glad_landcover_2020.lte(96)), 73)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(173)).And(glad_landcover_2020.lte(196)), 73)

# #vegetation
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(0)).And(glad_landcover_2020.lte(24)), 24)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(100)).And(glad_landcover_2020.lte(124)), 24)
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.eq(240)).Or(glad_landcover_2020.eq(249)), 24)

# #cropland
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(244)).And(glad_landcover_2020.lte(247)), 244)

# #water
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(200)).And(glad_landcover_2020.lte(211)), 200)
# glad_landcover_2020_main = glad_landcover_2020_main.where(glad_landcover_2020.eq(254), 200)

# #built up
# glad_landcover_2020_main = glad_landcover_2020_main.where((glad_landcover_2020.gte(250)).And(glad_landcover_2020.lte(253)), 250)


#### Get stable trees for 2020

In [141]:
glad_stable_tree_2020 = glad_landcover_2020_main.eq(25) #binary stable trees

glad_stable_tree_2020 = area_stats.set_scale_property_from_image(glad_stable_tree_2020,glad_landcover_2020)

glad_stable_tree_2020_area_km2 = area_stats.binary_to_area_km2(glad_stable_tree_2020)

glad_stable_tree_2020_area_km2 = area_stats.set_scale_property_from_image(glad_stable_tree_2020_area_km2,glad_landcover_2020)

In [142]:
# ##checks
# Map = geemap.Map()
# Map.addLayer(glad_stable_tree_2020,{'min':0,'max':1,'palette':["white","green"]},"glad_stable_tree_2020")
# Map

#### Fetch: ESRI 10m Annual Land Use Land Cover (2017-2022)¶
- Link: https://gee-community-catalog.org/projects/S2TSLULC/?h=esri

- Overview: Time series of annual global maps of land use and land cover (LULC). It currently has data from 2017-2021. The maps are derived from ESA Sentinel-2 imagery at 10m resolution. Each map is a composite of LULC predictions for 9 classes throughout the year in order to generate a representative snapshot of each year. This dataset was generated by Impact Observatory, who used billions of human-labeled pixels (curated by the National Geographic Society) to train a deep learning model for land classification. This map uses an updated model from the 10-class model and combines Grass(formerly class 3) and Scrub (formerly class 6) into a single Rangeland class (class 11). The original Esri 2020 Land Cover collection uses 10 classes (Grass and Scrub separate) and an older version of the underlying deep learning model. 

- Reference:  Karra, Kontgis, et al. “Global land use/land cover with Sentinel-2 and deep learning.” IGARSS 2021-2021 IEEE International Geoscience and Remote Sensing Symposium. IEEE, 2021.
- Legend (remapped values and hex code):
- 1	Water	#1A5BAB
- 2	Trees	#358221
- 3	Flooded Vegetation	#87D19E
- 4	Crops	#FFDB5C
- 5	Built Area	#ED022A
- 6	Bare Ground	#EDE9E4
- 7	Snow/Ice	#F2FAFF
- 8	Clouds	#C8C8C8
- 9	Rangeland	#C6AD8D

In [143]:
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS");

esri_lulc10_2020 = esri_lulc10.filterDate('2020-01-01','2020-12-31').map(
    lambda image:
    image.remap([1,2,4,5,7,8,9,10,11],
                [1,2,3,4,5,6,7,8,9])).mosaic()


In [144]:
esri_trees_2020 = esri_lulc10_2020.eq(2) #get trees    NB check flooded veg class

esri_trees_2020_area_km2 = area_stats.binary_to_area_km2(esri_trees_2020)

esri_trees_2020_area_km2 = area_stats.set_scale_property_from_image(esri_trees_2020_area_km2,esri_lulc10.first(),0,verbose=True)


template_band_index:  0
scale (m):  10


In [145]:
#checks
# Map = geemap.Map()
# Map.addLayer(esri_trees_2020,{'min':0,'max':1,'palette':["white","green"]})
# Map

#### Fetch: Tropical Moist Forest by JRC
Link: https:#  www.science.org/doi/10.1126/sciadv.abe1603 (paper); https://forobs.jrc.ec.europa.eu/static/tmf/TMF_DataUsersGuide.pdf (dataset description)  

Overview: The transition map shows the spatial distribution of the tropical moist forest at the end of
the year 2022. It depicts the sequential dynamics of changes by providing transition stages
from the first year of the monitoring period to the end of the year 2022 (undisturbed forest,
degradation, deforestation, regrowth, conversion to plantations) and subclasses for each
transition class (period of disturbance, age of regrowth, several types of forest, several
types of degradation and deforestation, change types within the mangroves and tree
plantations).

Reference: C. Vancutsem, F. Achard, J.-F. Pekel, G. Vieilledent, S. Carboni, D. Simonetti, J. Gallego,
L.E.O.C. Aragão, R. Nasi. Long-term (1990-2019) monitoring of forest cover changes in the
humid tropics. Science Advances 2021


Legend: 
- value 10. Undisturbed Tropical Moist Forest (TMF) 
- value 20. Degraded TMF 
- value 30. TMF regrowth 
- value 41. Deforested land - Forest converted to tree plantations 
- value 42. Deforested Land - Forest converted to water
- value 43. Deforested Land - Forest converted to other LC
- value 50. Ongoing deforestation or degradation (2019-2021)
- value 60. Permanent or Seasonal Water  (Pekel et al. 2016 & updates for years 2015-2021) 
- value 70. Other land cover (including afforestation)

In [146]:
JRC_TMF_transitions_raw = ee.ImageCollection('projects/JRC/TMF/v1_2021/TransitionMap_Subtypes')

JRC_TMF_transitions = JRC_TMF_transitions_raw.mosaic() ### NB check why 2021?

JRC_TMF_transitions_main = JRC_TMF_transitions.where((JRC_TMF_transitions.gte(10))and(JRC_TMF_transitions.lte(12)), 10) 
#JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(21))and(JRC_TMF_transitions.lte(26)), 20)
#JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(61))and(JRC_TMF_transitions.lte(62)), 20)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(31))and(JRC_TMF_transitions.lte(33)), 30)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(63))and(JRC_TMF_transitions.lte(64)), 30)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(81))and(JRC_TMF_transitions.lte(86)), 41)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(73))and(JRC_TMF_transitions.lte(74)), 42)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(41))and(JRC_TMF_transitions.lte(42)), 43)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(65))and(JRC_TMF_transitions.lte(66)), 43)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(51))and(JRC_TMF_transitions.lte(54)), 50)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.eq(67)),50)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(71))and(JRC_TMF_transitions.lte(72)), 60)
# JRC_TMF_transitions_main = JRC_TMF_transitions_main.where((JRC_TMF_transitions.gte(91)), 70)

##### Get undisturbed trees for 2020

In [147]:
JRC_TMF_undisturbed_2020 = JRC_TMF_transitions_main.eq(10) # undisturbed trees (binary)

#getting scale from first band of raw image and adding to properties of final composite
JRC_TMF_undisturbed_2020 = area_stats.set_scale_property_from_image(JRC_TMF_undisturbed_2020,JRC_TMF_transitions_raw.first())

#area (NB could be done at end in zonal stats)
JRC_TMF_undisturbed_2020_area_km2 = area_stats.binary_to_area_km2(JRC_TMF_undisturbed_2020)

JRC_TMF_undisturbed_2020_area_km2 = area_stats.set_scale_property_from_image(JRC_TMF_undisturbed_2020_area_km2,
                                                                  JRC_TMF_transitions_raw.first())

print (JRC_TMF_undisturbed_2020_area_km2.get("scale").getInfo())

30.000000000000004


In [148]:
# #checks
# Map = geemap.Map()
# Map.addLayer(JRC_TMF_undisturbed_2020,{'min':0,'max':1,'palette':["white","green"]},'JRC_TMF_undisturbed_2020')
# # m.addLayer(JRC_TMF_transitions_main.eq(10).randomVisualizer(),{},'TMF-simple')

# Map

#### Fetch: Global Forest Change 
Link: https://www.science.org/doi/10.1126/science.1244693 (paper); https://storage.googleapis.com/earthenginepartners-hansen/GFC-2022-v1.10/download.html (user notes v1.10)

Citation: Hansen, M. C., P. V. Potapov, R. Moore, M. Hancher, S. A. Turubanova, A. Tyukavina, D. Thau, S. V. Stehman, S. J. Goetz, T. R. Loveland, A. Kommareddy, A. Egorov, L. Chini, C. O. Justice, and J. R. G. Townshend. "High-Resolution Global Maps of 21st-Century Forest Cover Change." Science 342 (15 November): 850-53. 10.1126/science.1244693.

Overview: Results from time-series analysis of Landsat images in characterizing global forest extent and change from 2000 through 2022.


N.B. Next GFC update should give tree cover in 2020. Currently showing tree cover is for 2000 with loss pixels between 2001 and 2020 removed. 


In [149]:
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

gfc_treeCover_2000 = gfc.select(['treecover2000']) #get tree cover in 2000

gfc_loss_2001_2020 = gfc.select(['lossyear']).lte(20) # get loss pixels since 2000 and up to and including 2020

gfc_treeCover_2020 = gfc_treeCover_2000.where(gfc_loss_2001_2020.eq(1),0) # remove loss from original tree cover ot approximate remaining percentage cover


##### Get treecover in 2020

In [150]:
gfc_treeCover_2020_binary=gfc_treeCover_2020.gt(10) #FAO 10% definition...

## TO DO think about gain and 2010 cover

gfc_treeCover_2020_area_km2 = area_stats.binary_to_area_km2(gfc_treeCover_2020_binary)

gfc_treeCover_2020_area_km2 = area_stats.set_scale_property_from_image(gfc_treeCover_2020_area_km2,gfc)


In [151]:
##checks
# Map = geemap.Map()
# Map.addLayer(gfc_treeCover_2020_binary,{'min':0,'max':1,'palette':["blue","green"]})
# Map

#### Fetch: Primary humid forest GLAD/UMD
Link: https://developers.google.com/earth-engine/datasets/catalog/UMD_GLAD_PRIMARY_HUMID_TROPICAL_FORESTS_v1

Citation: Turubanova S., Potapov P., Tyukavina, A., and Hansen M. (2018) Ongoing primary forest loss in Brazil, Democratic Republic of the Congo, and Indonesia. Environmental Research Letters. https://doi.org/10.1088/1748-9326/aacd1c

Overview: Created by the UMD GLAD team. The primary forest extent was mapped for the year 2001 at a spatial resolution of 30 meters using globally acquired, free-of-charge, and consistently processed Landsat imagery

In [152]:
primary_HT_forests_2001_raw = ee.ImageCollection('UMD/GLAD/PRIMARY_HUMID_TROPICAL_FORESTS/v1')

#get band and mosaic
primary_HT_forests_2001 = primary_HT_forests_2001_raw.select("Primary_HT_forests").mosaic().selfMask();

#remove GFC loss pixels from 2001-2020 (as previous technique with GFC, above)
primary_HT_forests_2020 = primary_HT_forests_2001.where(gfc_loss_2001_2020.eq(1),0).selfMask()


In [153]:
primary_HT_forests_2020_area_km2 = area_stats.binary_to_area_km2(primary_HT_forests_2020)

primary_HT_forests_2020_area_km2 = area_stats.set_scale_property_from_image(primary_HT_forests_2020_area_km2,primary_HT_forests_2001_raw.first(),0,verbose=True)

# # checks
# Map = geemap.Map()

## Map.addLayer(Primary_HT_forests_2001, {'min': 0,'max': 1.0,'palette':['White",'Blue']}, 'Primary HT forests: 2001');
# Map.addLayer(Primary_HT_forests_2020, {'min': 0,'max': 1.0,'palette':['White",'Green']}, 'Primary HT forests: 2020');

# Map

template_band_index:  0
scale (m):  27.829872698318393


#### Fetch: JAXA Forest/non-forest maps


Links: https://www.eorc.jaxa.jp/ALOS/en/dataset/fnf_e.htm; https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF4

Citation:
Masanobu Shimada, Takuya Itoh, Takeshi Motooka, Manabu Watanabe, Shiraishi Tomohiro, Rajesh Thapa, and Richard Lucas, "New Global Forest/Non-forest Maps from ALOS PALSAR Data (2007-2010)", Remote Sensing of Environment, 155, pp. 13-31, December 2014. doi:10.1016/j.rse.2014.04.014.

Overview: The global forest/non-forest map (FNF) was produced the years 2017-2020. are generated by classifying the SAR image (backscattering coefficient) in the global 25m resolution PALSAR-2/PALSAR SAR mosaic so that strong and low backscatter pixels are assigned as "forest" and "non-forest", respectively. Here, "forest" is defined as the natural forest with the area larger than 0.5 ha and forest cover over 10%. This definition is the same as the Food and Agriculture Organization (FAO) definition. Since the radar backscatter from the forest depends on the region (climate zone), the classification of Forest/Non-Forest is conducted by using a region-dependent threshold of backscatter. The classification accuracy is checked by using in-situ photos and high-resolution optical satellite images.

Global 4-class
- 1 	 #00b200	Dense Forest
- 2 	 #83ef62	Non-dense Forest
- 3 	 #ffff99	Non-Forest
- 4 	 #0000ff	Water

In [154]:
JAXA_forestNonForest_raw = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF4');

JAXA_forestNonForest_2020 =  JAXA_forestNonForest_raw.filterDate('2020-01-01', '2020-12-31').select('fnf').mosaic();

#select all trees (could split these dense and non-dense, but hard to say how to group) 
JAXA_forestNonForest_2020_binary = JAXA_forestNonForest_2020.lte(2)

In [155]:
JAXA_forestNonForest_2020_area_km2 = area_stats.binary_to_area_km2(JAXA_forestNonForest_2020_binary)

JAXA_forestNonForest_2020_area_km2 = area_stats.set_scale_property_from_image(JAXA_forestNonForest_2020_area_km2,JAXA_forestNonForest_raw.first(),0,verbose=True)

template_band_index:  0
scale (m):  24.73766462072746


In [156]:
## checks
# Map = geemap.Map()

# Map.addLayer(JAXA_forestNonForest_2020, {'min': 1,'max': 4,'palette': ['00b200','83ef62','ffff99','0000ff']}, 'JAXA Forest/Non-Forest 2020 - 4 classes');

# Map.addLayer(JAXA_forestNonForest_2020_binary,{'min': 0,'max': 1.0,'palette':['White','Green']}, 'JAXA Forest/Non-Forest 2020');

# Map

#### Fetch: oil palm data
Link: https://developers.google.com/earth-engine/datasets/catalog/BIOPAMA_GlobalOilPalm_v1;  https://essd.copernicus.org/articles/13/1211/2021/

Citation: Adrià, Descals, Serge, Wich, Erik, Meijaard, David, Gaveau, Stephen, Peedell, & Zoltan, Szantoi. (2021, January 27). High resolution global industrial and smallholder oil palm map for 2019 (Version v1). Zenodo. doi:10.5281/zenodo.4473715

Overview: The dataset is a 10m global industrial and smallholder oil palm map for 2019. It covers areas where oil palm plantations were detected. The classified images are the output of a convolutional neural network based on Sentinel-1 and Sentinel-2 half-year composites.

In [157]:
# Import the dataset; a collection of composite granules from 2019.
oil_palm_descals_raw = ee.ImageCollection('BIOPAMA/GlobalOilPalm/v1');

# Select the classification band and mosaic all of the granules into a single image.
oil_palm_descals_mosaic = oil_palm_descals_raw.select('classification').mosaic();

# Visualisation only - not needed: create a mask to add transparency to non-oil palm plantation class pixels.
mask = oil_palm_descals_mosaic.neq(3);

mask = mask.where(mask.eq(0), 0.6); #not sure about this - from online (check)

oil_palm_descals_binary = oil_palm_descals_mosaic.lte(2)

In [158]:
oil_palm_descals_binary_area_km2 = area_stats.binary_to_area_km2(oil_palm_descals_binary)

oil_palm_descals_binary_area_km2 = area_stats.set_scale_property_from_image(oil_palm_descals_binary_area_km2,oil_palm_descals_raw.first(),0,verbose=True)

##checks
# Define visualization parameters.
# classificationVis = {
#   'min': 1,
#   'max': 3,
#   'palette': ['ff0000','ef00ff', '696969']
# };
# Map = geemap.Map()

# # Display the data on the map.
# Map.addLayer(oil_palm_descals_mosaic.updateMask(mask),
#              classificationVis, 'Oil palm plantation type', 1,1);
# Map.addLayer(oil_palm_descals_binary,
#              {'min':0,'max':1,'palette':["white","orange"]}, 'Oil palm plantation binary', 1,1);

# Map.setCenter(-3.0175, 5.2745,12);
# Map

template_band_index:  0
scale (m):  10


#### Fetch: World Database on Protected Areas (WDPA)
Link: www.protectedplanet.net

Citation: UNEP-WCMC and IUCN (year), Protected Planet: The World Database on Protected Areas (WDPA) [On-line], [insert month/year of the version used], Cambridge, UK: UNEP-WCMC and IUCN Available at: www.protectedplanet.net.

Overview: The World Database on Protected Areas (WDPA) is the most up-to-date and complete source of information on protected areas, updated monthly with submissions from governments, non-governmental organizations, landowners, and communities. It is managed by the United Nations Environment Programme's World Conservation Monitoring Centre (UNEP-WCMC) with support from IUCN and its World Commission on Protected Areas (WCPA).

WDPA User Manual. For details including methodologies,standards, data providers, metadata field definitions and descriptions, refer to the WDPA User Manual.
 

NB Filtering and preparation steps needs checking as based on use for protected area statistics
- date: this is using "current" data (it's updated monthly) - assuming if plot is in a protected area they need to know even if after 2020.
- point data: currently buffering by reported area where it exists
- removed sites: removed all but designated sites; removed Man and Biosphere reserves (for detail see: WDPA Manual)

TO DO: add OECMs - these are now being recognised as protected areas


In [159]:
wdpa_pnt = ee.FeatureCollection("WCMC/WDPA/current/points");

wdpa_poly = ee.FeatureCollection("WCMC/WDPA/current/polygons");

#apply filters and merge polygon with buffered points  
wdpa_filt = WDPA_prep.filterWDPA(wdpa_poly).merge(WDPA_prep.filterWDPA(wdpa_pnt).filter(ee.Filter.gt('REP_AREA', 0)).map(WDPA_prep.bufferByArea));
#turn into image (no crs etc set currently)
wdpa_overlap = wdpa_filt.reduceToImage(['STATUS_YR'],'min');  #make into raster - remove mask if want 0s

#make binary
wdpa_binary = wdpa_overlap.lt(2070).unmask()


##### Setting projection and resolution etc (based on GFC scale as template)

In [160]:
#reproject based on gfc data
crs_template = gfc.select(0).projection().crs().getInfo()

wdpa_binary_reproj = wdpa_binary.reproject(
  crs= crs_template,
  scale= area_stats.get_scale_from_image(gfc),
).int8()

protected_areas_WDPA_area_km2 = area_stats.binary_to_area_km2(wdpa_binary_reproj)

protected_areas_WDPA_area_km2 = area_stats.set_scale_property_from_image(protected_areas_WDPA_area_km2,gfc,0,verbose=True)


template_band_index:  0
scale (m):  27.829872698318393


In [161]:
# ##checks

# Map = geemap.Map()

# Map.addLayer(wdpa_filt,"","filtered wdpa_poly buff pnt",0,1)
# Map.addLayer(wdpa_binary_reproj, {min:0, max:1, 'palette':['white','purple']},'wdpa_binary_reproj',1,1); #visualise raster - with count of overlaps
# Map.addLayer(protected_areas_WDPA_area_km2, {min:0, max:10, 'palette':['blue','red']},'protected_areas_WDPA_area_km2',1,1); #visualise raster - min status yr (only useful for timeseries work)

# Map

#### Fetch: Key Biodiversity Areas (KBAs)
Link: https://www.keybiodiversityareas.org/kba-data/request; 
https://www.keybiodiversityareas.org/termsofservice

Citation: BirdLife International ([year e.g. 2023]). The World Database of Key Biodiversity Areas. Developed by the KBA Partnership: BirdLife International, International Union for the Conservation of Nature, Amphibian Survival Alliance, Conservation International, Critical Ecosystem Partnership Fund, Global Environment Facility, Re:wild, NatureServe, Rainforest Trust, Royal Society for the Protection of Birds, Wildlife Conservation Society and World Wildlife Fund. Available at www.keybiodiversityareas.org. [Accessed (please insert date of download dd/mm/yyyy)].

Overview: Key Biodiversity Areas, which are among the most incredible and diverse places on Earth for nature, from deserts to the middle of the ocean, are sites of global importance to the planet’s overall health and the persistence of biodiversity. The Key Biodiversity Area Partnership - an ambitious partnership of 13 global conservation organizations - is helping prevent the rapid loss of biodiversity by supporting nationally led efforts to identify these places on the planet that are critical for the survival of unique plants and animals, and the ecological communities they comprise.

By mapping these most important sites on Earth, and providing information about the wildlife living there, private industry, governments and other stakeholders can make the best decisions about how to manage that land (or waters), where to avoid development, and how best to conserve and protect the animals and plants for which the sites are so important.

NB
- Useful flag as these areas typically become protected (large overlap with existing protected areas whcih take a w hwile to be designated) 
- The last populations of species are found in a subset of KBAs (i.e., Alliance for Zero Extinction (AZE) sites)
- KBAs have rigourous inclusion criteria and typically have active management for protecting species they contain

In [162]:
kbas_2023_poly = ee.FeatureCollection("projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/raw/KBAsGlobal_2023_March_01_POL");##uploaded - may need rights

kba_2023_overlap = kbas_2023_poly.reduceToImage(['SitRecID'],'count').selfMask()  #make into raster - remove mask if want 0s

kba_2023_binary = kba_2023_overlap.gte(0)

##### Setting projection and resolution etc (based on GFC scale as template)

In [163]:
#reproject based on gfc data
crs_template = gfc.select(0).projection().crs().getInfo()

kba_2023_binary_reproj = kba_2023_binary.reproject(
  crs= crs_template,
  scale= area_stats.get_scale_from_image(gfc),
).int8()

kba_2023_area_km2 = area_stats.binary_to_area_km2(kba_2023_binary_reproj)

kba_2023_area_km2 = area_stats.set_scale_property_from_image(kba_2023_area_km2,gfc,0,verbose=True)



template_band_index:  0
scale (m):  27.829872698318393


In [164]:
# # ##checks
# Map = geemap.Map()
# # Map.addLayer(kbas_2023_poly,"",'kbas_2023_poly',0,1); 
# # Map.addLayer(kba_2023_overlap, {'min':0, 'max':5, 'palette':['blue','red']},'kba_overlap',0,1); #visualise raster - with count of overlaps
# Map.addLayer(kba_2023_binary, {'min':0, 'max':1, 'palette':['white','black']},'kba_binary',1,1); #binary raster

# Map

#### Fetch: RADD forest disturbance alert

Link: http://radd-alert.wur.nl; https://gee-community-catalog.org/projects/radd/

Citation: Reiche J, Mullissa A, Slagter B, Gou Y, Tsendbazar N-E, Odongo-Braun C, Vollrath A, Weisse MJ, Stolle F, Pickens A, Donchyts G, Clinton N, Gorelick N, and Herold M (2021). https://doi.org/10.1088/1748-9326/abd0a8

Overview:
RADD - RAdar for Detecting Deforestation - Near real-time disturbances in humid tropical forest based on Sentinel-1 at 10m spatial scale. 
NB only primary humid tropical forest of South America (13 countries), Central America (6 countries), Africa (25 countries), insular Southeast Asia (5 countries) and Pacific (1 country).
A new forest disturbance alert is triggered based on a single observation from the latest Sentinel-1 C-band radar image. Subsequent observations are used to iteratively update the forest disturbance probability, increase confidence and confirm or reject the alert. Alerts are confirmed within a maximum 90-day period if the forest disturbance probability is above 97.5% (high confidence alerts). Unconfirmed alerts (low confidence alerts) are provided for forest disturbance probabilities above 85%. The date of the alert is set to the date of the image that first triggered the alert.


Approach for deforestations risk proxy: run stats for areas around each input feature and 5km buffer. 
Select only recent forest loss (i.e. in the last 2 years) based on todays date.

Possible data alternatives: use GFC loss (not as recent); use combination of loss and alerts; use off-shelf deforestation risk product (no buffer needed for this).

Instead of buffer and recency, use actual rates such as here https://code.earthengine.google.com/ab1e640b81a107b796718f285a56422a


In [165]:
# Getting today's date
ee_now =ee.Date(datetime.now())#.format()
# print(ee_now.getInfo())

# Calculate the start date
start_date = ee_now.advance(how_many_days_back, "day")

# Needs to be in yyDDD format and needs to be a number, so need to parse too
start_date_yyDDD = ee.Number.parse(start_date.format('yyDDD'))

print("start_date_yyDDD", start_date_yyDDD.getInfo())


start_date_yyDDD 21316


In [166]:
# Define the Image Collection
radd = ee.ImageCollection('projects/radar-wur/raddalert/v1')

# Forest baseline (from Primart HT forests data)
forest_baseline = ee.Image(radd.filterMetadata('layer', 'contains', 'forest_baseline')
    .mosaic())
 
# Latest RADD alert
latest_radd_alert = ee.Image(radd.filterMetadata('layer', 'contains', 'alert')
    .sort('system:time_end', False)
    .mosaic())

# Update mask for RADD alert to be within primary forest (TO CHECK maybe unnecessry step)
latest_radd_alert_masked = latest_radd_alert.select('Alert').updateMask(forest_baseline)

# Mask confirmed alerts #TO CHECK do we want to be more conservative?
latest_radd_alert_masked_confirmed = latest_radd_alert_masked.eq(3).selfMask()

# Update mask for confirmed alerts by date
latest_radd_alert_confirmed_recent = latest_radd_alert.select('Date').gte(start_date_yyDDD).selfMask()


In [167]:
latest_radd_alert_confirmed_recent_area_km2 = area_stats.binary_to_area_km2(latest_radd_alert_confirmed_recent)

latest_radd_alert_confirmed_recent_area_km2 = area_stats.set_scale_property_from_image(latest_radd_alert_confirmed_recent_area_km2,radd.first(),0,verbose=True)

template_band_index:  0
scale (m):  10


In [168]:
# #checks
# Map = geemap.Map()

# # # # Add Forest baseline layer to the map
# Map.addLayer(forest_baseline, {'palette': ['darkgreen']}, 'Forest baseline', 0, 1)

# Map.addLayer(latest_radd_alert_confirmed_recent,
#     {'min': 0, 'max': 1, 'palette': ['white', 'red']},
#     'RADD alert masked confirmed - recent (i.e., filtered by date)', 1, 1)

# Map

### 2. Fetch some fields (public)

In [169]:
CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
           'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f',
           'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c',
'97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7',
       '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45',
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

all_geo_ids= CIV_ids+GHA_ids+IDN_ids

print (all_geo_ids)

['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918', 'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f', 'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3', 'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f', '88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c', '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb', 'c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7', '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45', '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']


#### Transform geometries into a feature collection
NB Currently not working due to session issues

In [170]:
# roi = geo_id_list_to_feature_collection(all_geo_ids)

# #checks 
# print ("Count of geo ids in list: ", len(all_geo_ids))
# print ("Count of features in FeatureCollection: ", roi.size().getInfo())

##### Alternative feature collection: create random polygons
select random points inside administrative boundaries and buffer them 

In [171]:
def create_random_points_in_polys(feature): #to tidy
    """ creates random points within either a polygon or a feature collection NB relies upon some globals being set currently"""
    return ee.FeatureCollection.randomPoints(region = feature.geometry(max_error), points = number_of_points, seed=seed, maxError=10)

admin_boundaries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2").filter(ee.Filter.inList("ADM0_NAME",["Indonesia","Ghana"]))##.filter(ee.Filter.lt("Shape_Area",1000))

random_collection = admin_boundaries.randomColumn(seed = seed).sort('random').limit(number_of_boundaries)

if points_per_admin_boundary:
    random_points = random_collection.map(create_random_points_in_polys).flatten()
else:
    random_points = create_random_points_in_polys(random_collection)
    
random_buffers = random_points.map(lambda feature: feature.buffer(buffer_distance_meters,100)) ##buffer by distance in meters

roi= random_buffers.map(lambda feature: feature.set(geo_id_column,(feature.get("system:index"))))
# roi= random_buffers.map(set_geo_id_from_system_index) ## add surrogate "geo_id" for formatting

#checks
if verbose:
    print("number of admin regions",random_collection.size().getInfo())
    print("number of countries",random_collection.aggregate_array("ADM0_NAME").distinct().length().getInfo())
    print("number of countries",random_collection.aggregate_array("ADM0_NAME").distinct().getInfo())
    print("number of buffers created",random_buffers.size().getInfo())
    # geemap.ee_to_pandas(roi)###create random points and buffer them


number of admin regions 20
number of countries 2
number of countries ['Indonesia', 'Ghana']
number of buffers created 30


In [172]:
# ## checks
# Map=geemap.Map()
# Map.addLayer(roi, "", 'roi', 1, 1)
# Map

#### Feature prep
- Create additional buffer zones for deforestation risk 
- Add area property to feature(s) 
- Select only columns of interest

In [173]:
roi = area_stats.add_area_km2_property_to_feature_collection(roi,geometry_area_column)

roi  = roi.select([geometry_area_column,geo_id_column]) ##select only fields of interest

roi_alerts_buffer = roi.map(lambda feature: 
        feature.buffer(local_alerts_buffer_radius,max_error_alert_buff))

## to check HOW TO HANDLE ALERT BUFFER - best ignore until know how/if alerts will be used like this
###roi_alerts_buffer = roi_alerts_buffer.map(add_area_km2_property_to_feature).select([geometry_area_column,geo_id_column])

geemap.ee_to_pandas(roi_alerts_buffer)
geemap.ee_to_pandas(roi)

,Geo_id,Geometry_area_km2
0,0,0.684923
1,1,0.684923
2,2,0.684923
3,3,0.684923
4,4,0.684923
5,5,0.684923
6,6,0.684923
7,7,0.684923
8,8,0.684923
9,9,0.684923


In [174]:
# ## checks
# Map=geemap.Map()
# Map.addLayer(roi_alerts_buffer, "", 'roi_alerts_buffer', 1, 1)
# Map

#### Set properties for which images need special treatment

In [175]:
#deforestation alerts
# set property so run stats for a buffer around site; 
# and show presence only as output 
latest_radd_alert_confirmed_recent_area_km2 = latest_radd_alert_confirmed_recent_area_km2.setMulti(
    {"alerts_buffer":1,"presence_only_flag":1})

#important sites: 1) protected areas and 2) KBAs (likely future protectred areas) 
# show presence only as output 
protected_areas_WDPA_area_km2 = protected_areas_WDPA_area_km2.set("presence_only_flag",1)

kba_2023_area_km2 = kba_2023_area_km2.set("presence_only_flag",1)


#### Create dictionary of images and image names
- prep for reduceRegions statistics so name of datasets/image is added to area stats
- sets "system:index" property of each image
- result is an image collection with 

In [180]:
image_names_dict0 = {"GFC_Treecover_2020":gfc_treeCover_2020_area_km2,
               "TMF_undisturbed_forest_2020":JRC_TMF_undisturbed_2020_area_km2,
                "ESRI_Trees_2020":esri_trees_2020_area_km2,
                "GLAD_Stable_Tree 2020":glad_stable_tree_2020_area_km2,
                "Oil_Palm_Descals": oil_palm_descals_binary_area_km2,
                "Local_RADD_alerts":latest_radd_alert_confirmed_recent_area_km2,
                "Protected_area":protected_areas_WDPA_area_km2,
                "Key_Biodiversity_Area": kba_2023_area_km2,
                  "Primary_HT_forests_2020": primary_HT_forests_2020_area_km2,
                   "JAXA_Forest_non_forest_2020":JAXA_forestNonForest_2020_area_km2
              }

## can be cleaned up...

#create empty dictionary to be populated
image_names_dict={} 
    
#set image names ("system:index") from keys in dictionary, and store as new one
for i in range(len(image_names_dict0)):
    dataset_name = (list(image_names_dict0.keys())[i]) #get dataset name
    image = (list(image_names_dict0.values())[i]) #get image
    updated_image=image.set("system:index",dataset_name) #set dataset name as image name i.e., "system:index"
    instance={dataset_name:updated_image} #combine
    image_names_dict.update(instance) #update into new dictionary

del image_names_dict0 # remove old dictionary

#make into a new image collection
images_IC = ee.ImageCollection(list(image_names_dict.values()))
                      
##checks
print ("number of images: ",len(image_names_dict))

images_IC

number of images:  10


#### Compute statistics

Calculating zonal statistics for continuous data (e.g tree cover) within polygon(s)

step 1: save as combined CSV in long format NB needs speeding up e.g., reduceRegions and dataframes

In [192]:
# get the start time
st = time.time()
print ("processing stats...")


#get stats for roi (except alerts)
fc_stats_combined = area_stats.reduceStatsIC(roi,
                                  images_IC.filter(ee.Filter.neq("alerts_buffer",1)),
                                  reducer_choice)# all but alerts
#get stat for buffer (alerts only)
fc_stats_combined_buffer = area_stats.reduceStatsIC(roi_alerts_buffer,
                                         images_IC.filter(ee.Filter.eq("alerts_buffer",1)),
                                         reducer_choice) #alerts only

#combine stats from roi and buffer
fc_stats_combined_all = fc_stats_combined.merge(fc_stats_combined_buffer) # combining alerts with others into one feature collection


# convert to Pandas Dataframe
df_combined = geemap.ee_to_pandas(fc_stats_combined_all) # limit of 5000 (unlikely to need more but i have code for it if needed)

# export dataframe to csv
df_combined.to_csv(path_or_buf=out_file_long,header=True,index=False)

# get the execution time
elapsed_time = time.time() - st

print('Total execution time:', elapsed_time, 'seconds')

#checks
df_combined

processing stats...
Total execution time: 4.624518156051636 seconds


,dataset_name,sum,count,Geo_id,Geometry_area_km2
0,GFC_Treecover_2020,0.129839,946,0,0.684923
1,GFC_Treecover_2020,0.666981,949,1,0.684923
2,GFC_Treecover_2020,0.464474,952,2,0.684923
3,GFC_Treecover_2020,0.681785,952,3,0.684923
4,GFC_Treecover_2020,0.669723,957,4,0.684923
...,...,...,...,...,...
295,Local_RADD_alerts,0.000000,0,25,0.684923
296,Local_RADD_alerts,0.000000,0,26,0.684923
297,Local_RADD_alerts,0.000000,0,27,0.684923
298,Local_RADD_alerts,0.000000,0,28,0.684923


##### Reformat
- long to wide
- convert to proportions 
- set presence only flags

In [193]:
#add proprtion column
df_combined["proportion"] = (df_combined["sum"]/df_combined[geometry_area_column])

#convert to wide format (one row per geo_id)
df_wide_format = df_combined.pivot_table(index=[geo_id_column],columns=['dataset_name'],values=['proportion'])

#tidy unwanted headers etc
tidy_dataframe_after_pivot(df_wide_format) #runs in place so no need to assign

#list images with with presence_only_flag property 
flag_list = images_IC.filter(ee.Filter.eq("presence_only_flag",1)).aggregate_array("system:index").getInfo()

#convert pos results values to "True" for specific columns
for column in flag_list: 
    df_wide_format[column]=np.where(df_wide_format[column]>0,"True","-")

#tidy output - decimal places
df_wide_format = df_wide_format.round(decimals=2, out=None)

#tidy (i.e. remove underscores)
df_wide_format.columns = df_wide_format.columns.str.replace('_', ' ')

#export wide format csv
df_wide_format.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)

#checks
flag_list
df_wide_format


output csv:  /home/sepal-user/fdap/stats_wide_format_continuous_data.csv


,ESRI Trees 2020,GFC Treecover 2020,GLAD Stable Tree 2020,JAXA Forest non forest 2020,Key Biodiversity Area,Local RADD alerts,Oil Palm Descals,Primary HT forests 2020,Protected area,TMF undisturbed forest 2020
Geo_id,,,,,,,,,,
0,1.00,0.19,0.99,0.00,-,-,0.99,0.00,-,0.00
1,1.00,0.97,1.00,0.98,True,True,0.00,0.70,-,0.70
10,1.00,0.98,1.00,0.85,True,True,0.00,0.96,True,0.93
11,0.96,0.76,0.93,0.00,-,-,0.67,0.00,-,0.00
12,0.94,0.50,0.94,0.06,-,True,0.59,0.00,-,0.00
13,0.00,0.06,0.77,0.00,-,-,1.00,0.06,-,0.00
14,1.00,1.00,1.00,0.98,-,True,0.00,1.00,True,0.99
15,1.00,0.96,1.00,1.00,True,True,0.00,0.94,True,0.91
16,1.00,0.90,0.99,0.38,-,-,0.00,0.00,-,0.01


##### Mapping over image collection with reduce regions

#### Display layers

NB add legends


ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

In [121]:
Map = geemap.Map()

In [122]:
vis_params = {min:0,max:1,'palette':["white","green"]}

Map.addLayer(gfc_treeCover_2020_binary,vis_params,'gfc_treeCover_2020_binary')
Map.addLayer(esri_trees_2020,vis_params,"esri_trees_2020")
Map.addLayer(esri_trees_2020,vis_params,"esri_trees_2020")
Map.addLayer(glad_stable_tree_2020,vis_params,'glad_stable_tree_2020')
Map.addLayer(JRC_TMF_undisturbed_2020,vis_params,'JRC_TMF_undisturbed_2020')


# m.addLayer(JRC_TMF_transitions.eq(10).randomVisualizer(),{},'TMF-simple')

Map.addLayer(roi,{},'roi ')

Map.addLayer(roi_alerts_buffer,{},'roi buffer zone')

# m.centerObject(roi.first())

single_feature = roi_alerts_buffer.filter(ee.Filter.eq(geo_id_column,7)).first()
Map.centerObject(roi_alerts_buffer)

In [123]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### Alternative stats


In [124]:
# # Define the class names
# class_names = ['TMF_undis', 'TMF_degrad', 'TMF_regrow', 'Defor_plantation', 'Defor_water', 'Defor_land','Defor_active','water','other_land']

# # Create a function to count pixels for each class
# def count_pixels(image, roi):
#     counts = image.reduceRegion(
#         reducer=ee.Reducer.frequencyHistogram(),
#         geometry=roi,
#         scale=90,
#         maxPixels=1e9
#     )
    
#     # Convert pixel counts to a feature with properties
#     feature = ee.Feature(None, counts)
    
#     return feature

# # Apply the count_pixels function to the classified image
# pixel_counts_feature = count_pixels(tmf, roi)

# # Create a feature collection with a single feature
# fc_pix = ee.FeatureCollection([pixel_counts_feature])


# fc_pix_df = geemap.ee_to_pandas(fc_pix)##needs more work
# fc_pix_df


### Logout (protected)

In [125]:
res = session.get(asset_registry_base + "/logout")
print(res.json())
res = session.get(user_registry_base + "/logout", cookies=session.cookies)
session.headers.clear()

NameError: name 'session' is not defined

### Checking if Logged out correctly

In [ ]:
# Confirming the logout from Asset Registry by requesting a Protected route
req_body = {
    "latitude": 31.47704430446457,
    "longitude": 74.37510786779589
}
res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
print(res.json())

### Get all Domains (public)

In [ ]:
# Fetching all the domains from the User Registry
res = session.get(asset_registry_base + "/domains")
print(res.json())